## Problem 1 [Waiting]
Use the inverse CDF method to generate independent samples, Xi, of the exponential
random variable with average waiting time of 0.2 time units. Evaluate the quality of your
generator with goodness of fit tests.

## Problem 2 [Counting]
Each exponential random sample represents the waiting time until an event occurs.
Implement a routine to count the number of events that occur in 1 unit of time. Generate
such counts for 1000 separate unit time intervals. How are these counts distributed? Justify
your answer



## Problem 3 [Networking]
Given n=50 people at a networking event. Imagine that any given unordered pair of two
people connect at random and independently with probability p.
Generate and plot sample networks for each value of p={0.02, 0.09}. Briefly discuss the
structure of these sample graphs.



## Problem 4 [Networking at larger scale]
Count the number of connections (edges) to a vertex (node) in one of your sample
networks from Problem 3. This statistic is the degree of that vertex. The vertex degrees for
all vertices in a random network are random. Vertex degree is a binomially distributed
statistic for small networks.
Plot a histogram of vertex degrees for one of your sample networks in Problem 3. Repeat
for the other sample network.
Generate a network with (n, p) = (250, 0.08) and plot the histogram of its vertex degrees.
What distribution fits this data? Justify your claim with any probabilistic limit laws you
know. 